<div style="text-align: right"><b> Ana Valentina López Chacón </b></div>
<div style="text-align: right"><b> Traducción Automática, 2024 - 2025 </b></div>

# **Python a Ingles: 1-shot LLAMA-2 Baseline**

In [1]:
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q datasets jiwer tqdm pandas
!pip install -q torch torchvision torchaudio
!pip install -q evaluate
!pip install -q transformers==4.47.0
!pip install -q accelerate==0.27.0
!pip install -q peft
!pip install -q bitsandbytes==0.45.0
!pip install -q sacrebleu
!pip install -q unbabel-comet
!pip install -q rouge_score

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset

dataset = load_dataset("code_x_glue_ct_code_to_text", "python")

dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/26.7k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/144M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/147M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/18.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/251820 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13914 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/14918 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 251820
    })
    validation: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 13914
    })
    test: Dataset({
        features: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url'],
        num_rows: 14918
    })
})

In [3]:
dataset['train'].features

{'id': Value(dtype='int32', id=None),
 'repo': Value(dtype='string', id=None),
 'path': Value(dtype='string', id=None),
 'func_name': Value(dtype='string', id=None),
 'original_string': Value(dtype='string', id=None),
 'language': Value(dtype='string', id=None),
 'code': Value(dtype='string', id=None),
 'code_tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'docstring': Value(dtype='string', id=None),
 'docstring_tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'sha': Value(dtype='string', id=None),
 'url': Value(dtype='string', id=None)}

In [4]:
print(len(dataset['train']))
print(len(dataset['validation']))
print(len(dataset['test']))

251820
13914
14918


In [5]:
seed = 42

train_dataset = dataset['train'].shuffle(seed=seed).select(range(200))
validation_dataset = dataset['validation'].shuffle(seed=seed).select(range(500))
test_dataset = dataset['test'].shuffle(seed=seed).select(range(500))

print("Training dataset size:", len(train_dataset))
print("Validation dataset size:", len(validation_dataset))
print("Test dataset size:", len(test_dataset))

Training dataset size: 200
Validation dataset size: 500
Test dataset size: 500


In [6]:
train_dataset[0]['code']

'def metapolicy(request, permitted, domains=None):\n    """\n    Serves a cross-domain policy which can allow other policies\n    to exist on the same domain.\n\n    Note that this view, if used, must be the master policy for the\n    domain, and so must be served from the URL ``/crossdomain.xml`` on\n    the domain: setting metapolicy information in other policy files\n    is forbidden by the cross-domain policy specification.\n\n    **Required arguments:**\n\n    ``permitted``\n        A string indicating the extent to which other policies are\n        permitted. A set of constants is available in\n        ``flashpolicies.policies``, defining acceptable values for\n        this argument.\n\n    **Optional arguments:**\n\n    ``domains``\n        A list of domains from which to allow access. Each value may\n        be either a domain name (e.g., ``example.com``) or a wildcard\n        (e.g., ``*.example.com``). Due to serious potential security\n        issues, it is strongly recommen

In [7]:
train_dataset[0]["docstring"]

'Serves a cross-domain policy which can allow other policies\n    to exist on the same domain.\n\n    Note that this view, if used, must be the master policy for the\n    domain, and so must be served from the URL ``/crossdomain.xml`` on\n    the domain: setting metapolicy information in other policy files\n    is forbidden by the cross-domain policy specification.\n\n    **Required arguments:**\n\n    ``permitted``\n        A string indicating the extent to which other policies are\n        permitted. A set of constants is available in\n        ``flashpolicies.policies``, defining acceptable values for\n        this argument.\n\n    **Optional arguments:**\n\n    ``domains``\n        A list of domains from which to allow access. Each value may\n        be either a domain name (e.g., ``example.com``) or a wildcard\n        (e.g., ``*.example.com``). Due to serious potential security\n        issues, it is strongly recommended that you not use wildcard\n        domain values.'

In [8]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) N
Token is valid (permission: fineGrained).
The token `token_llama` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `token_

In [9]:
from transformers import AutoTokenizer

max_tok_length = 100

checkpoint = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(
    checkpoint, use_auth_token=True,
    padding=True,
    pad_to_multiple_of=8,
    truncation=True,
    max_length=max_tok_length,
    padding_side='left',
    )
tokenizer.pad_token = "[PAD]"

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:810: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [10]:
def preprocess_function(sample):
    model_inputs = tokenizer(
        sample["code"],
        text_target = sample["docstring"],
        )
    return model_inputs

In [11]:
test_dataset_filtered = test_dataset.filter(lambda x: len(preprocess_function(x)["input_ids"]) <= max_tok_length and len(preprocess_function(x)["labels"]) <= max_tok_length,
                                               desc=f"Discarding source and target sentences with more than {max_tok_length} tokens")

Discarding source and target sentences with more than 100 tokens:   0%|          | 0/500 [00:00<?, ? examples/…

In [12]:
print("Test dataset size:", len(test_dataset_filtered))

Test dataset size: 99


In [13]:
tokenized_test_dataset = test_dataset_filtered.map(preprocess_function, batched=True)

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

In [14]:
dic = {}
for sample in tokenized_test_dataset:
    sample_length = len(sample['input_ids'])
    if sample_length not in dic:
        dic[sample_length] = 1
    else:
        dic[sample_length] += 1

for i in range(1,max_tok_length+1):
    if i in dic:
        print(f"{i:>2} {dic[i]:>3}")

43   1
44   1
45   1
48   1
49   2
50   3
51   2
53   2
54   1
56   1
57   1
59   1
60   3
61   3
62   1
63   2
64   1
65   4
66   2
67   3
69   3
70   2
71   5
72   1
73   2
74   5
77   3
80   2
81   1
82   1
83   2
84   3
85   3
86   4
88   3
89   2
90   2
91   2
92   3
94   3
95   1
96   1
97   3
98   1
99   4
100   1


In [15]:
src = "code"
tgt = "docstring"
task_prefix = f"Generate the English {tgt} from the following Python {src}:\n"
num_shots = 1
shots = ""
s = ""

prefix_tok_len = len(tokenizer.encode(f"{task_prefix}{shots}{src}: {s} = {tgt}: "))
shot_tok_len   = len(tokenizer.encode(f"{src}: {s} = {tgt}: {s}\n"))
max_tok_len = prefix_tok_len
max_tok_len += num_shots * (shot_tok_len + 2 * max_tok_length)
max_tok_len += max_tok_length

sample = tokenized_test_dataset.shuffle(seed=seed).select(range(num_shots))
for s in sample: shots += f"{src}: {s['code']} = {tgt}: {s['docstring']}\n"

def preprocess4test_function(sample):
    inputs = [f"{task_prefix}{shots}{src}: {s} = {tgt}: " for s in sample["code"]]
    model_inputs = tokenizer(
        inputs,
        max_length=max_tok_len,
        truncation=True,
        return_tensors="pt",
        padding=True)
    return model_inputs

In [16]:
sample = tokenized_test_dataset.select(range(2))
model_input = preprocess4test_function(sample)
print(model_input)
print(tokenizer.batch_decode(model_input['input_ids']))

{'input_ids': tensor([[    1,  7338,  1461,   278,  4223,  1574,  1807,   515,   278,  1494,
          5132,   775, 29901,    13,   401, 29901,   822,   679, 29918,  5750,
           919, 29879, 29898,  1311,  1125,    13,  4706,  9995, 11609,   263,
          1051,   411,  1353,   310,  3415, 18186,  7090,  3454,   639,  3370,
         21277,  3233,  1213, 15945,    13,  4706,   736,   518,  1311, 29889,
         16202, 29961,  8173, 29918,  5563, 22322, 29923, 18118,  1783, 29918,
          2798,  2033, 29914,  1311, 29889,  4102, 29918,  6229, 29918, 19790,
            13, 18884,   363,  7090, 29918,  5563,   297,  3464, 29898,  2435,
         29898,  1311, 29889, 23523,  1839, 14834, 21277, 25901, 29962,   353,
          1574,  1807, 29901,  7106,   263,  1051,   411,  1353,   310,  3415,
         18186,  7090,  3454,   639,  3370, 21277,  3233, 29889,    13,   401,
         29901,   822,  2048, 29918,  5552, 29879, 29898,  1311, 29892,  2967,
         29918,  5552, 29879, 29892,  

In [17]:
preprocessed_test_dataset = tokenized_test_dataset.map(preprocess4test_function, batched=True)

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

In [18]:
for sample in preprocessed_test_dataset.select(range(2)):
    print(sample['input_ids'])
    print(sample['attention_mask'])
    print(sample['labels'])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 7338, 1461, 278, 4223, 1574, 1807, 515, 278, 1494, 5132, 775, 29901, 13, 401, 29901, 822, 679, 29918, 5750, 919, 29879, 29898, 1311, 1125, 13, 4706, 9995, 11609, 263, 1051, 411, 1353, 310, 3415, 18186, 7090, 3454, 639, 3370, 21277, 3233, 1213, 15945, 13, 4706, 736, 518, 1311, 29889, 16202, 29961, 8173, 29918, 5563, 22322, 29923, 18118, 1783, 29918, 2798, 2033, 29914, 1311, 29889, 4102, 29918, 6229, 29918, 19790, 13, 18884, 363, 7090, 29918, 5563, 297, 3464, 29898, 2435, 29898, 1311, 29889, 23523, 1839, 14834, 21277, 25901, 29962, 353, 1574, 1807, 29901, 7106, 263, 1051, 411, 1353, 310, 3415, 18186, 7090, 3454, 639, 3370, 21277, 3233, 29889, 13, 401, 29901, 822, 2048, 29918, 5552, 29879, 29898, 1311, 29892, 2967, 29918, 5552, 29879, 29892, 4805, 29918, 5552, 29879, 29922, 8516, 1125, 13, 4706, 9995, 8893, 385, 5352, 8600, 1213, 15945, 13, 4706, 12421, 29879, 353, 2967, 29918, 5552, 29879, 29889

In [19]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [20]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    quantization_config=quantization_config,
    torch_dtype=torch.bfloat16,
)


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [21]:
from transformers import GenerationConfig

generation_config = GenerationConfig.from_pretrained(
    checkpoint,
    )

print(generation_config)

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 0,
  "temperature": 0.6,
  "top_p": 0.9
}



### **Evaluación**

In [22]:
test_batch_size = 16
batch_tokenized_test = preprocessed_test_dataset.batch(test_batch_size)

Batching examples:   0%|          | 0/99 [00:00<?, ? examples/s]

In [23]:
number_of_batches = len(batch_tokenized_test["input_ids"])
output_sequences = []
for i in range(number_of_batches):
    output_batch = model.generate(
        generation_config=generation_config,
        input_ids=torch.tensor(batch_tokenized_test["input_ids"][i]).cuda(),
        attention_mask=torch.tensor(batch_tokenized_test["attention_mask"][i]).cuda(),
        max_length = max_tok_len,
        num_beams=1,
        do_sample=False,)
    output_sequences.extend(output_batch)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [24]:
decoded_preds = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)

In [25]:
references = tokenizer.batch_decode(preprocessed_test_dataset["labels"], skip_special_tokens=True)

In [26]:
sources = test_dataset_filtered['code']

In [27]:
from whisper.normalizers.basic import BasicTextNormalizer

normalizer = BasicTextNormalizer()

decoded_preds_clean = [normalizer(text) for text in decoded_preds]
references_clean = [normalizer(text) for text in references]
sources_clean = [normalizer(text) for text in sources]

In [28]:
from evaluate import load

metric = load("sacrebleu")

In [29]:
result = metric.compute(predictions=decoded_preds_clean, references=references_clean)
print(f'BLEU score: {result["score"]:.1f}')

BLEU score: 6.9


In [30]:
from evaluate import load

rouge_metric = load("rouge")
comet_metric = load("comet")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/3.38k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/9.69k [00:00<?, ?B/s]

model.ckpt:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [31]:
comet_score = comet_metric.compute(predictions = decoded_preds_clean, references = references_clean, sources = sources_clean)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [32]:
print(f"COMET: {comet_score['mean_score'] * 100:.2f} %")

COMET: 53.01 %


In [33]:
rouge_score = rouge_metric.compute(predictions = decoded_preds_clean, references = references_clean)

In [34]:
print(f"Rouge 1: {rouge_score['rouge1'] * 100:.2f} %")
print(f"Rouge 2: {rouge_score['rouge2'] * 100:.2f} %")
print(f"Rouge L: {rouge_score['rougeL'] * 100:.2f} %")

Rouge 1: 14.41 %
Rouge 2: 13.01 %
Rouge L: 14.37 %
